In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import escalateclient
import importlib

In [2]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
username = 'vshekar'
password = 'copperhead123'
client = escalateclient.ESCALATEClient(server_url, username, password)

In [7]:
neilson_lab_response = client.get_or_create(endpoint='actor', data={'description': 'Neilson Lab', 'person__isnull': True, 'systemtool__isnull': True})[0]

{'description': 'Neilson Lab', 'person__isnull': True, 'systemtool__isnull': True}
http://localhost:8000/api/actor/?description=Neilson+Lab&person__isnull=True&systemtool__isnull=True
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [8]:
template_response = client.get_or_create(endpoint='experiment-template', data={'description': 'Neilson Experiment', 'ref_id': 'workflow_1', 'lab': neilson_lab_response['url']})[0]

{'description': 'Neilson Experiment', 'ref_id': 'workflow_1', 'lab': '9df76199-42b8-44ae-8b7b-ce41d48ccbf6'}
http://localhost:8000/api/experiment-template/?description=Neilson+Experiment&ref_id=workflow_1&lab=9df76199-42b8-44ae-8b7b-ce41d48ccbf6
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [3]:
reagent_templates = {
            "Reagent 1 - Nitride Precursors": ["nitride"],
            "Reagent 2 - Oxide Precursors": ["oxide"],
            "Reagent 3 - Fluxes": ["flux"],
        }

reagent_template_responses = client.create_reagent_templates(data=reagent_templates)

{'description': 'Reagent 1 - Nitride Precursors'}
http://localhost:8000/api/reagent-template/?description=Reagent+1+-+Nitride+Precursors
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'nitride'}
http://localhost:8000/api/material-type/?description=nitride
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Reagent 1 - Nitride Precursors: nitride', 'reagent_template': '6c110c88-9f11-4284-91ca-d98e65a35959', 'material_type': 'bb4ac792-378c-4c1b-8b2e-626b2ad3ab27'}
http://localhost:8000/api/reagent-material-template/?description=Reagent+1+-+Nitride+Precursors%3A+nitride&reagent_template=6c110c88-9f11-4284-91ca-d98e65a35959&material_type=bb4ac792-378c-4c1b-8b2e-626b2ad3ab27
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Reagent 2 - Oxide Precursors'}
http://localhost:8000/api/reagent-template/?description=Reagent+2+-+Oxide+Precursors
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'

In [5]:
zero_g_value = {"value": 0, "unit": "g", "type": "num"}
zero_g_data = {"description": "Zero g", "nominal_value": zero_g_value, "actual_value": zero_g_value,}
zero_g_response = client.get_or_create(endpoint='default-values', data=zero_g_data)[0]

nitride = client.get(endpoint='reagent-template', 
                     resource_id=reagent_template_responses['Reagent 1 - Nitride Precursors']['uuid'], 
                     related_endpoint='reagent-material-template')[0]
oxide = client.get(endpoint='reagent-template', 
                     resource_id=reagent_template_responses['Reagent 2 - Oxide Precursors']['uuid'], 
                     related_endpoint='reagent-material-template')[0]
flux = client.get(endpoint='reagent-template', 
                     resource_id=reagent_template_responses['Reagent 3 - Fluxes']['uuid'], 
                     related_endpoint='reagent-material-template')[0]


data = {"description": "amount",
        "reagent_material_template": nitride['url'], # Associating it with nitride
        "default_value": zero_g_response['url']}
client.get_or_create('reagent-material-value-template', data=data)

data = {"description": "amount",
        "reagent_material_template": oxide['url'], # Associating it with nitride
        "default_value": zero_g_response['url']}
client.get_or_create('reagent-material-value-template', data=data)

data = {"description": "amount",
        "reagent_material_template": flux['url'], # Associating it with nitride
        "default_value": zero_g_response['url']}
client.get_or_create('reagent-material-value-template', data=data)

{'description': 'Zero g'}
http://localhost:8000/api/default-values/?description=Zero+g
GET: OK. Found 1 results
{}
http://localhost:8000/api/reagent-template/6c110c88-9f11-4284-91ca-d98e65a35959/reagent-material-template/
GET: OK. Found 1 results
{}
http://localhost:8000/api/reagent-template/ee6a5374-a2a2-4063-b8dd-837c599793fe/reagent-material-template/
GET: OK. Found 1 results
{}
http://localhost:8000/api/reagent-template/8764b6c7-3f35-40c0-a837-92a6d5443ce2/reagent-material-template/
GET: OK. Found 1 results
{'description': 'amount', 'reagent_material_template': '8374da94-1178-4086-b4dd-217f219c7d1c', 'default_value': 'b8fcdb40-9879-40f6-a71d-10eda9a84fe6'}
http://localhost:8000/api/reagent-material-value-template/?description=amount&reagent_material_template=8374da94-1178-4086-b4dd-217f219c7d1c&default_value=b8fcdb40-9879-40f6-a71d-10eda9a84fe6
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'amount', 'reagent_material_template': '8e1c9a76-fea7-41be-9

[{'url': 'http://localhost:8000/api/reagent-material-value-template/b681b487-80b7-4ead-8eae-b27a31d81125/',
  'uuid': 'b681b487-80b7-4ead-8eae-b27a31d81125',
  'edocs': [],
  'tags': [],
  'notes': [],
  'add_date': '2022-03-02T16:40:10.527831',
  'mod_date': '2022-03-02T16:40:10.527856',
  'description': 'amount',
  'status': None,
  'reagent_material_template': 'http://localhost:8000/api/reagent-material-template/4a162eae-e4bc-489d-965c-e33c1fa50fd4/',
  'default_value': 'http://localhost:8000/api/default-values/b8fcdb40-9879-40f6-a71d-10eda9a84fe6/'}]

In [9]:
action_sequence_names = ["Perform experiment"]
asn_responses = {}
for sequence_number, asn in enumerate(action_sequence_names):
    data = {'description': asn}
    # Create action sequence
    asn_responses[asn] = client.get_or_create(endpoint='action-sequence', data=data)[0]
    # Associate with experiment template
    data = {'experiment_action_sequence_seq': sequence_number,
            'experiment_template': template_response['url'],
            'action_sequence': asn_responses[asn]['url']}
    client.get_or_create(endpoint='experiment-action-sequence', data=data)

{'description': 'Perform experiment'}
http://localhost:8000/api/action-sequence/?description=Perform+experiment
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'experiment_template': 'ba0ea00b-dd89-47fb-923a-faa9e418bf2e', 'action_sequence': '3d0a84e0-0853-4afb-a006-182853380a03'}
http://localhost:8000/api/experiment-action-sequence/?experiment_template=ba0ea00b-dd89-47fb-923a-faa9e418bf2e&action_sequence=3d0a84e0-0853-4afb-a006-182853380a03
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [10]:
action_parameter_def = {
            "bring_to_temperature": [("temperature", {"type": "num", "unit": "degC", "value": 0.0})],
            "dwell": [("duration", {"type": "num", "unit": "seconds", "value": 0.0})],
            "cool_to_temperature": [("temperature", {"type": "num", "unit": "degC", "value": 0.0})],
        }
# Convenience function to add action defs and their parameter definitions
action_defs, parameter_defs = client.create_action_parameters(data=action_parameter_def)


{'description': 'bring_to_temperature'}
http://localhost:8000/api/action-def/?description=bring_to_temperature
GET: OK. Found 1 results
[('temperature', {'type': 'num', 'unit': 'degC', 'value': 0.0})]
{'description': 'temperature'}
http://localhost:8000/api/parameter-def/?description=temperature
GET: OK. Found 2 results
{'description': 'dwell'}
http://localhost:8000/api/action-def/?description=dwell
GET: OK. Found 1 results
[('duration', {'type': 'num', 'unit': 'seconds', 'value': 0.0})]
{'description': 'duration'}
http://localhost:8000/api/parameter-def/?description=duration
GET: OK. Found 2 results
{'description': 'cool_to_temperature'}
http://localhost:8000/api/action-def/?description=cool_to_temperature
GET: OK. Found 0 results
POST: OK, returning new resource dict
[('temperature', {'type': 'num', 'unit': 'degC', 'value': 0.0})]
{'description': 'temperature'}
http://localhost:8000/api/parameter-def/?description=temperature
GET: OK. Found 2 results
